In [12]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time
import bs4 as bs
import urllib.request
import nltk
import heapq
import re

In [13]:
# url definition
url = "https://www.dailymail.co.uk"

In [14]:
# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
coverpage_news = soup1.find_all('h2', class_='linkro-darkred')
len(coverpage_news)

83

In [15]:
coverpage_news[4]

<h2 class="linkro-darkred">
        <a href="/news/article-9207367/Covid-UK-Cases-fall-29-week-21-088-deaths-fall-3-7.html" itemprop="url">UK Covid cases fall 29% in a week to 21,088 and deaths fall by 3.7% with another 587 fatalities recorded as public health experts say impact of vaccine should be seen in 'two weeks' </a>
    </h2>

In [16]:
number_of_articles = 5

In [17]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
        
    # Getting the link of the article
    link = url + coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('p', class_='mol-para-with-font')
    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(body)):
        paragraph = body[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    # Removing special characters
    final_article = re.sub("\\xa0", "", final_article)
        
    news_contents.append(final_article)

In [18]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents 
    })

# df_show_info
df_show_info = pd.DataFrame(
    {'Article Title': list_titles,
     'Article Link': list_links})

In [19]:
df_features

,Article Content
0,Ministers sparked confusion over the summer ho...
1,A Dorset beach was last night battered by a br...
2,Ireland's prime minister lashed out at the Eur...
3,A refusal to go on ventilators is putting crit...
4,Coronavirus cases and deaths fell again today ...


In [20]:
df_show_info

,Article Title,Article Link
0,Confusion as Matt Hancock tells Britons they c...,https://www.dailymail.co.uk/news/article-92071...
1,Sea defences are breached and cars are carried...,https://www.dailymail.co.uk/news/article-92066...
2,Irish PM Micheal Martin blasts the EU saying p...,https://www.dailymail.co.uk/news/article-92068...
3,Critically ill Covid patients are dying unnece...,https://www.dailymail.co.uk/news/article-92069...
4,"UK Covid cases fall 29% in a week to 21,088 an...",https://www.dailymail.co.uk/news/article-92073...


In [31]:
def summary_of_article1(article):
    sentence_scores = {}
 
    scraped_data = urllib.request.urlopen(article)
    
    article = scraped_data.read()

    parsed_article = bs.BeautifulSoup(article,'lxml')

    paragraphs = parsed_article.find_all('p')

    article_text = ""

    for p in paragraphs:
        article_text += p.text
        
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)
    
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    
    sentence_list = nltk.sent_tokenize(article_text)
    
    stopwords = nltk.corpus.stopwords.words('french')
    
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 50:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    return summary

In [32]:
# empty list for article summaries
articles_summaries = []
# getting the summary for each article
for i in np.arange(0, number_of_articles):
    articles_summaries.append(summary_of_article1(list_links[i]))

In [33]:
articles_summaries[0]

"The Prime Minister told the European Commission president that plans to stop 3.5 million doses from the Pfizer factory in Belgium the UK risked preventing people from receiving the second injection, forcing it into a hurried climbdown. Following the PM's diplomatic victory, Britain yesterday recorded a daily record for first-dose jabs – 487,756 – to bring the total to almost 8.4 million. Speaking to BBC One's Andrew Marr Show, Dr Hopkins said: 'What we would like to see is a divergence in the case rate in the over-70s and over-80s who have been vaccinated from the younger age groups, to show that they are declining faster. 'Any releases that we have will have to happen very slowly, very cautiously, watching and waiting as we go, with a two-week period to watch and see the impact of that relaxation because it takes that to see what's happening in the population. 'And they did that, in my view, without anywhere near the understanding of the Good Friday Agreement, of the sensitivity of t

In [34]:
articles_summaries[1]

'The Environment Agency had issued a flood warning for the area, forecasting sea spray overtopping along the coast as a result of high tides and strong winds and urging people to be careful along beaches, promenades and coastal footpaths and roads due to the danger of big waves. \'In the colder air to the north of this system the rainfall will readily turn to snow or even freezing rain, affecting a large part of England and Wales north of the M4 corridor. \'However, rain is likely to turn to sleet and snow across Wales and central England during the morning and then some parts of south and south-east England before it clears southwards again. Met Office Chief Meteorologist Andy Page said yesterday: \'Over the next few days we continue to see a division between milder conditions in the South West with much colder air to the north and east. There are 250 alerts across the country where flooding is possibleI just hate the expression "the white stuff" when ...Published by Associated Newspa

In [35]:
articles_summaries[2]

"'We are only four weeks into the operation of the Protocol, there are bound to be teething problems but I do acknowledge the need for engagement here on all sides, between the European Union, the United Kingdom and the Irish Government, and the Northern Ireland Executive. However Mr Martin laid some blame at the door of AstraZeneca, saying there is a 'strong sense across Europe' that the Anglo-Sewdish jab manufacturer has not delivered on commitments around the vaccination. 'Following the PM's diplomatic victory, Britain yesterday recorded a daily record for first-dose jabs – 487,756 – to bring the total to almost 8.4 million. ''I don't think politicians should pronounce on the efficacy or otherwise of vaccines,' the Irish leader said. 'Published by Associated Newspapers LtdPart of the Daily Mail, The Mail on Sunday & Metro Media Group There are issues there that we have to fine-tune and work out, but essentially I think there are positives there medium term for Northern Ireland in te

In [36]:
articles_summaries[3]

"'She told the newspaper that colleagues had watched patients die because they had refused to go on ventilators during the second wave of the pandemic. She added: 'If you get to that point where you are being offered a ventilator, if you refuse there is a 100 per cent chance you are going to die. 'They get sicker and sicker and sicker, then you have a situation when they become semi-conscious, so you can't have an informed conversation. 'But if you say yes it's a 40 to 50 per cent chance. It's the Covid that's killing you.' 'We have to honour our patients' wishes."

In [37]:
articles_summaries[4]

"But with Rishi Sunak mulling increases to capital gains tax to pay for the massive £400billion blackhole in public spending accrued during the pandemic and warnings that the economy could take a decade to recover, Tory MPs are likely to be rattled by the new graphs. The announcement follows reports that SAGE has advised the government to keep social distancing measures in place until the end of the year amid fears that reopening the country too quickly will prompt a third wave of infections. Speaking to BBC One's Andrew Marr Show, Dr Hopkins said: 'What we would like to see is a divergence in the case rate in the over-70s and over-80s who have been vaccinated from the younger age groups, to show that they are declining faster. The second wave has seen bigger numbers of coronavirus patients in hospitals, with 21,684 patients during the peak of the first wave of the pandemic last April. Also today, intensive care doctors said the coronavirus crisis in hospitals has 'stabilised' and staf